In [ ]:
# pip install openai pypdf2 pdfplumber

In [7]:
import PyPDF2
import os
from openai import OpenAI

In [8]:
client = OpenAI(
    api_key='YOUR_API_KEY'
    )

In [9]:
# Functions

# Function to extract the text from a PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = '\n'.join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

# Function to send text to ChatGPT and get the summary
def summarize_text(text):
  completion = client.chat.completions.create(
    model='gpt-4o-mini',
    store=True,
    messages=[
      {'role': 'system', 'content':'Make a summary from this text: '},
      {'role': 'user', 'content': text}
    ]
  )
  response = completion.choices[0].message.content.strip()
  return response

# Function to split the text if necessary
def split_text(text, max_length):
    parts = []
    while len(text) > max_length:
        part = text[:max_length]
        last_period = part.rfind('.')
        if last_period == -1:
            last_period = max_length
        parts.append(text[:last_period + 1])
        text = text[last_period + 1:]
    parts.append(text)
    return parts

In [11]:
# Path of PDF file
pdf_path = '/content/A ARTE DA GUERRA - ok.pdf'

# Character limit to send (ChatGPT has a token limit)
characters_limit = 470000

# Text extration
full_text = extract_text_from_pdf(pdf_path)
print(f'Extracted text: ({len(full_text)} characters).')

# If the text is longer than the limit, we divide it into parts
if len(full_text) > characters_limit:
    text_parts = split_text(full_text, characters_limit)
    summaries = []

    print(f'Big text! Divided in {len(text_parts)} parts...')

    for i, part in enumerate(text_parts):
        print(f'Sending part {i+1}/{len(text_parts)} to OpenAI...')
        summary = summarize_text(part)
        summaries.append(summary)

    # Final summary combining all parts
    final_summary = summarize_text('\n'.join(summaries))

else:
    print('Sending text to OpenAI...')
    final_summary = summarize_text(full_text)

# Save the summary in a separated file
text_name = pdf_path.split('/')[-1].split('.pdf')[0]
with open(f'{text_name}_summary.txt', 'w', encoding='utf-8') as f:
    f.write(final_summary)

print(f'Summary generated and save in "{text_name}_summary.txt!"')

Extracted text: (88941 characters).
Sending text to OpenAI...
Summary generated and save in "A ARTE DA GUERRA - ok_summary.txt!"
